# Exercises week 43

**October 18-25, 2024**

Date: **Deadline is Friday October 25 at midnight**


# Overarching aims of the exercises this week

The aim of the exercises this week is to train the neural network you implemented last week.

To train neural networks, we use gradient descent, since there is no analytical expression for the optimal parameters. This means you will need to compute the gradient of the cost function wrt. the network parameters. And then you will need to implement some gradient method.

You will begin by computing gradients for a network with one layer, then two layers, then any number of layers. Keeping track of the shapes and doing things step by step will be very important this week.

We recommend that you do the exercises this week by editing and running this notebook file, as it includes some checks along the way that you have implemented the neural network correctly, and running small parts of the code at a time will be important for understanding the methods. If you have trouble running a notebook, you can run this notebook in google colab instead(https://colab.research.google.com/drive/1FfvbN0XlhV-lATRPyGRTtTBnJr3zNuHL#offline=true&sandboxMode=true), though we recommend that you set up VSCode and your python environment to run code like this locally.

First, some setup code that you will need.


In [8]:
import autograd.numpy as np  # We need to use this numpy wrapper to make automatic differentiation work later
from autograd import grad, elementwise_grad
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score


# Defining some activation functions
def ReLU(z):
    return np.where(z > 0, z, 0)


# Derivative of the ReLU function
def ReLU_der(z):
    return np.where(z > 0, 1, 0)


def sigmoid(z):
    return 1 / (1 + np.exp(-z))


def mse(predict, target):
    return np.mean((predict - target) ** 2)

# Exercise 1 - Understand the feed forward pass

**a)** Complete last weeks' mandatory exercises if you haven't already.


# Exercise 2 - Gradient with one layer using autograd

For the first few exercises, we will not use batched inputs. Only a single input vector is passed through the layer at a time.

In this exercise you will compute the gradient of a single layer. You only need to change the code in the cells right below an exercise, the rest works out of the box. Feel free to make changes and see how stuff works though!


**a)** If the weights and bias of a layer has shapes (10, 4) and (10), what will the shapes of the gradients of the cost function wrt. these weights and this bias be?


Answer: If we use the OLS cost function we will be (50, 1) since we have to take the partial derivative of all the components of the bias and the weights components.

**b)** Complete the feed_forward_one_layer function. It should use the sigmoid activation function. Also define the weigth and bias with the correct shapes.


In [9]:
def feed_forward_one_layer(W, b, x):
    z = W @ x + b
    a = sigmoid(z)
    return a


def cost_one_layer(W, b, x, target):
    predict = feed_forward_one_layer(W, b, x)
    return mse(predict, target)


x = np.random.rand(2)

target = np.random.rand(3)

W = np.random.rand(3,2)
b = np.random.rand(3)

**c)** Compute the gradient of the cost function wrt. the weigth and bias by running the cell below. You will not need to change anything, just make sure it runs by defining things correctly in the cell above. This code uses the autograd package which uses backprogagation to compute the gradient!


In [10]:
autograd_one_layer = grad(cost_one_layer, [0, 1])
W_g, b_g = autograd_one_layer(W, b, x, target)
print(W_g, b_g)

[[0.02501038 0.04440928]
 [0.0163832  0.02909056]
 [0.02788055 0.04950565]] [0.08563488 0.05609563 0.09546226]


# Exercise 3 - Gradient with one layer writing backpropagation by hand

Before you use the gradient you found using autograd, you will have to find the gradient "manually", to better understand how the backpropagation computation works. To do backpropagation "manually", you will need to write out expressions for many derivatives along the computation.


We want to find the gradient of the cost function wrt. the weight and bias. This is quite hard to do directly, so we instead use the chain rule to combine multiple derivatives which are easier to compute.

$$
\frac{dC}{dW} = \frac{dC}{da}\frac{da}{dz}\frac{dz}{dW}
$$

$$
\frac{dC}{db} = \frac{dC}{da}\frac{da}{dz}\frac{dz}{db}
$$


**a)** Which intermediary results can be reused between the two expressions?

Both expression make use of these two partial derivatives, so we can reuse these two

$$
\frac{dC}{da} , \frac{da}{dz}
$$


**b)** What is the derivative of the cost wrt. the final activation? You can use the autograd calculation to make sure you get the correct result. Remember that we compute the mean in mse.


In [16]:
z = W @ x + b
a = sigmoid(z)

predict = a


def mse_der(predict, target):
    n = max(predict.shape)
    return 2*(predict - target)*(1/n)


print(mse_der(predict, target))

cost_autograd = grad(mse, 0)
print(cost_autograd(predict, target))

[0.52786491 0.53707936 0.47898141]
[0.52786491 0.53707936 0.47898141]


**c)** What is the expression for the derivative of the sigmoid activation function? You can use the autograd calculation to make sure you get the correct result.


In [17]:
def sigmoid_der(z):
    return sigmoid(z)*(1-sigmoid(z))


print(sigmoid_der(z))

sigmoid_autograd = elementwise_grad(sigmoid, 0)
print(sigmoid_autograd(z))

[0.1648543  0.156597   0.20226993]
[0.1648543  0.156597   0.20226993]


**d)** Using the two derivatives you just computed, compute this intermetidary gradient you will use later:

$$
\frac{dC}{dz} = \frac{dC}{da}\frac{da}{dz}
$$


In [18]:
dC_da = mse_der(predict,target)
dC_dz = dC_da*sigmoid_der(z)

**e)** What is the derivative of the intermediary z wrt. the weight and bias? What should the shapes be? The one for the weights is a little tricky, it can be easier to play around in the next exercise first. You can also try computing it with autograd to get a hint.

For the weights we will have,
$$
\frac{dz}{dw} =  \begin{bmatrix}
\mathbf{x}^{T}\\
\vdots  \} n\\
\mathbf{x}^{T}
\end{bmatrix}
$$
in which will have the same shape as the weight-matrix. We assume that x (the input) is a column vector Thus the above expression will have the shape of (mxn) i.e. the same as the original W, where n is the number of features in the input vector.

For the bias we will have a shape equvivalent to the bias.
$$
\frac{dz}{db} =  \mathbf{1}
$$



**f)** Now combine the expressions you have worked with so far to compute the gradients! Note that you always need to do a feed forward pass while saving the zs and as before you do backpropagation, as they are used in the derivative expressions


In [42]:
dC_da = dC_da
dC_dz = dC_dz


dz_dW = np.ones(W.shape)*x

n = max(b.shape)
dz_db = np.ones(n)

dC_dW = np.diag(dC_dz) @ dz_dW
dC_db = dC_dz * dz_db 

print(f'Weight gradient:\n {dC_dW} \n Bias gradinet {dC_db}')


Weight gradient:
 [[0.02541515 0.045128  ]
 [0.02456357 0.04361591]
 [0.02829565 0.0502427 ]] 
 Bias gradinet [0.0870208  0.08410502 0.09688354]


You should get the same results as with autograd.


In [43]:
W_g, b_g = autograd_one_layer(W, b, x, target)
print(f'Weight gradient:\n {W_g} \n Bias gradinet {b_g}')

Weight gradient:
 [[0.02541515 0.045128  ]
 [0.02456357 0.04361591]
 [0.02829565 0.0502427 ]] 
 Bias gradinet [0.0870208  0.08410502 0.09688354]


# Exercise 4 - Gradient with two layers writing backpropagation by hand


Now that you have implemented backpropagation for one layer, you have found most of the expressions you will need for more layers. Let's move up to two layers.


In [44]:
x = np.random.rand(2)
target = np.random.rand(4)

W1 = np.random.rand(3, 2)
b1 = np.random.rand(3)

W2 = np.random.rand(4, 3)
b2 = np.random.rand(4)

layers = [(W1, b1), (W2, b2)]

In [49]:
z1 = W1 @ x + b1
a1 = sigmoid(z1)

z2 = W2 @ a1 + b2
a2 = sigmoid(z2)

predict2 = a2

We begin by computing the gradients of the last layer, as the gradients must be propagated backwards from the end.

**a)** Compute the gradients of the last layer, just like you did the single layer in the previous exercise.


In [66]:
dC_da2 = mse_der(predict2,target)
dC_dz2 = dC_da2 * sigmoid_der(z2)

dz2_dW2 = np.ones(W2.shape)*a1

n2 = max(b2.shape)
dz2_db2 = np.ones(n2)

dC_dW2 = np.diag(dC_dz2) @ dz2_dW2
dC_db2 = dC_dz2 * dz2_db2 



To find the derivative of the cost wrt. the activation of the first layer, we need a new expression, the one furthest to the right in the following.

$$
\frac{dC}{da_1} = \frac{dC}{dz_2}\frac{dz_2}{da_1}
$$

**b)** What is the derivative of the second layer intermetiate wrt. the first layer activation? (First recall how you compute $z_2$)

$$
\frac{dz_2}{da_1} = W_2
$$


**c)** Use this expression, together with expressions which are equivelent to ones for the last layer to compute all the derivatives of the first layer.

$$
\frac{dC}{dW_1} = \frac{dC}{da_1}\frac{da_1}{dz_1}\frac{dz_1}{dW_1}
$$

$$
\frac{dC}{db_1} = \frac{dC}{da_1}\frac{da_1}{dz_1}\frac{dz_1}{db_1}
$$


In [104]:
dC_da1 = dC_dz2 @ W2
da1_dz1 = sigmoid_der(z1)



In [89]:
#dC_dz1 = dC_da1 * da1_dz1

dz1_dW1 = np.ones(W1.shape)*x
n1 = max(b1.shape)
dz1_db1 = np.ones(b1.shape)



In [95]:
dC_dW1 = np.diag((dC_da1 * da1_dz1)) @ dz1_dW1
dC_db1 = dC_da1 * da1_dz1 * dz1_db1

In [103]:
print(f'Cost derivative weights first: \n {dC_dW1} \n Cost derivative bias: \n {dC_db1} \n \n \n')
print(f'Cost derivative weights second: \n {dC_dW2} \n Cost derivatve bias \n {dC_db2}')

Cost derivative weights first: 
 [[0.00043089 0.00147988]
 [0.00237623 0.00816113]
 [0.00024261 0.00083324]] 
 Cost derivative bias: 
 [0.00153585 0.00846977 0.00086475] 
 
 

Cost derivative weights second: 
 [[-0.00729567 -0.00616818 -0.00759789]
 [ 0.00582073  0.00492118  0.00606185]
 [ 0.03517548  0.02973934  0.0366326 ]
 [-0.0084737  -0.00716414 -0.00882472]] 
 Cost derivatve bias 
 [-0.00945244  0.00754148  0.04557415 -0.01097872]


**d)** Make sure you got the same gradient as the following code which uses autograd to do backpropagation.


In [97]:
def feed_forward_two_layers(layers, x):
    W1, b1 = layers[0]
    z1 = W1 @ x + b1
    a1 = sigmoid(z1)

    W2, b2 = layers[1]
    z2 = W2 @ a1 + b2
    a2 = sigmoid(z2)

    return a2

In [98]:
def cost_two_layers(layers, x, target):
    predict = feed_forward_two_layers(layers, x)
    return mse(predict, target)


grad_two_layers = grad(cost_two_layers, 0)
grad_two_layers(layers, x, target)

[(array([[0.00043089, 0.00147988],
         [0.00237623, 0.00816113],
         [0.00024261, 0.00083324]]),
  array([0.00153585, 0.00846977, 0.00086475])),
 (array([[-0.00729567, -0.00616818, -0.00759789],
         [ 0.00582073,  0.00492118,  0.00606185],
         [ 0.03517548,  0.02973934,  0.0366326 ],
         [-0.0084737 , -0.00716414, -0.00882472]]),
  array([-0.00945244,  0.00754148,  0.04557415, -0.01097872]))]

**e)** How would you use the gradient from this layer to compute the gradient of an even earlier layer? Would the expressions be any different?

The expression would basically be the same, we use ', 
$$\frac{\partial C_{n+1}}{\partial W_{n+1}} ,
\frac{\partial C_{n+1}}{\partial b_{n+1}}
$$

In order to find 

$$\frac{\partial C_{n}}{\partial W_{n}} ,
\frac{\partial C_{n}}{\partial b_{n}}
$$

Where to only thing we need from the previous layer is
$$\frac{\partial z_{n+1}}{\partial a_{n}}
$$
Otherwiese the process will be similar

# Exercise 5 - Gradient with any number of layers writing backpropagation by hand


Well done on getting this far! Now it's time to compute the gradient with any number of layers.

First, some code from the general neural network code from last week. Note that we are still sending in one input vector at a time. We will change it to use batched inputs later.


In [203]:
def create_layers(network_input_size, layer_output_sizes):
    layers = []

    i_size = network_input_size
    for layer_output_size in layer_output_sizes:
        W = np.random.randn(layer_output_size, i_size)
        b = np.random.randn(layer_output_size)
        layers.append((W, b))

        i_size = layer_output_size
    return layers


def feed_forward(input, layers, activation_funcs):
    a = input
    for (W, b), activation_func in zip(layers, activation_funcs):
        z = W @ a + b
        a = activation_func(z)
    return a


def cost(layers, input, activation_funcs, target):
    predict = feed_forward(input, layers, activation_funcs)
    return mse(predict, target)

You might have already have noticed a very important detail in backpropagation: You need the values from the forward pass to compute all the gradients! The feed forward method above is great for efficiency and for using autograd, as it only cares about computing the final output, but now we need to also save the results along the way.

Here is a function which does that for you.


In [204]:
def feed_forward_saver(input, layers, activation_funcs):
    layer_inputs = []
    zs = []
    a = input
    for (W, b), activation_func in zip(layers, activation_funcs):
        layer_inputs.append(a)
        z = W @ a + b
        a = activation_func(z)

        zs.append(z)

    return layer_inputs, zs, a

**a)** Now, complete the backpropagation function so that it returns the gradient of the cost function wrt. all the weigths and biases. Use the autograd calculation below to make sure you get the correct answer.


In [330]:
def backpropagation(
    input, layers, activation_funcs, target, activation_ders, cost_der=mse_der
):
    layer_inputs, zs, predict = feed_forward_saver(input, layers, activation_funcs)
    print(f'Prediction is \n {predict}')

    layer_grads = [() for layer in layers]

    # We loop over the layers, from the last to the first
    for i in reversed(range(len(layers))):
        layer_input, z, activation_der = layer_inputs[i], zs[i], activation_ders[i]
        

        if i == len(layers) - 1:
            # For last layer we use cost derivative as dC_da(L) can be computed directly
            dC_da = cost_der(predict,target)
            print('Last')
        else:
            # For other layers we build on previous z derivative, as dC_da(i) = dC_dz(i+1) * dz(i+1)_da(i)
            (W, b) = layers[i + 1]
            dC_da =  dC_dz @ W
            print('Hidden')

        da_dz = activation_der(z)
        dC_dz = dC_da*da_dz
        
        
        dC_db = dC_da*da_dz

        dz_dW = np.tile(layer_input, (max(dC_db.shape), 1))
        dC_dW =   np.diag(dC_da * da_dz) @ dz_dW

        layer_grads[i] = (dC_dW, dC_db)

    return layer_grads

In [331]:
np.random.seed(1)

network_input_size = 2
layer_output_sizes = [3,4]
activation_funcs = [sigmoid, ReLU]
activation_ders = [sigmoid_der, ReLU_der]

layers = create_layers(network_input_size, layer_output_sizes)

x = np.random.rand(network_input_size)
target = np.random.rand(4) #Remember to change this to 4

In [332]:
layer_grads = backpropagation(x, layers, activation_funcs, target, activation_ders)
layer_grads

Prediction is 
 [0.53052074 0.94506556 0.         0.67702684]
Last
(4, 3)
(4,)
 [[-0.15828642 -0.04060292 -0.0660155 ]
 [ 0.27273946  0.0699619   0.1137497 ]
 [ 0.          0.          0.        ]
 [ 0.22900866  0.05874427  0.09551118]]
Hidden
(3, 2)
(3,)
 [[-0.00065147 -0.0028267 ]
 [-0.00421466 -0.01828724]
 [ 0.01073804  0.04659191]]


[(array([[-0.00065147, -0.0028267 ],
         [-0.00421466, -0.01828724],
         [ 0.01073804,  0.04659191]]),
  array([-0.0063111 , -0.04082944,  0.10402452])),
 (array([[-0.15828642, -0.04060292, -0.0660155 ],
         [ 0.27273946,  0.0699619 ,  0.1137497 ],
         [ 0.        ,  0.        ,  0.        ],
         [ 0.22900866,  0.05874427,  0.09551118]]),
  array([-0.18903738,  0.32572571, -0.        ,  0.27349913]))]

In [320]:
cost_grad = grad(cost, 0)
cost_grad(layers, x, activation_funcs, target)

[(array([[-0.00065147, -0.0028267 ],
         [-0.00421466, -0.01828724],
         [ 0.01073804,  0.04659191]]),
  array([-0.0063111 , -0.04082944,  0.10402452])),
 (array([[-0.15828642, -0.04060292, -0.0660155 ],
         [ 0.27273946,  0.0699619 ,  0.1137497 ],
         [ 0.        ,  0.        ,  0.        ],
         [ 0.22900866,  0.05874427,  0.09551118]]),
  array([-0.18903738,  0.32572571,  0.        ,  0.27349913]))]

# Exercise 6 - Batched inputs

Make new versions of all the functions in exercise 5 which now take batched inputs instead. See last weeks exercise 5 for details on how to batch inputs to neural networks. You will also need to update the backpropogation function.


# Exercise 7 - Training


**a)** Complete exercise 6 and 7 from last week, but use your own backpropogation implementation to compute the gradient.

**b)** Use stochastic gradient descent with momentum when you train your network.


# Exercise 8 (Optional) - Object orientation

Passing in the layers, activations functions, activation derivatives and cost derivatives into the functions each time leads to code which is easy to understand in isoloation, but messier when used in a larger context with data splitting, data scaling, gradient methods and so forth. Creating an object which stores these values can lead to code which is much easier to use.

**a)** Write a neural network class. You are free to implement it how you see fit, though we strongly recommend to not save any input or output values as class attributes, nor let the neural network class handle gradient methods internally. Gradient methods should be handled outside, by performing general operations on the layer_grads list using functions or classes separate to the neural network.

We provide here a skeleton structure which should get you started.


In [ ]:
class NeuralNetwork:
    def __init__(
        self,
        network_input_size,
        layer_output_sizes,
        activation_funcs,
        activation_ders,
        cost_fun,
        cost_der,
    ):
        pass

    def predict(self, inputs):
        # Simple feed forward pass
        pass

    def cost(self, inputs, targets):
        pass

    def _feed_forward_saver(self, inputs):
        pass

    def compute_gradient(self, inputs, targets):
        pass

    def update_weights(self, layer_grads):
        pass

    # These last two methods are not needed in the project, but they can be nice to have! The first one has a layers parameter so that you can use autograd on it
    def autograd_compliant_predict(self, layers, inputs):
        pass

    def autograd_gradient(self, inputs, targets):
        pass